### 132 데이터베이스

### 관계형데이터베이스
- SQL Alchemy
  - SQLote
  - MySQL

### NoSQL
- DBM memcach 
  - Key-value
- MonoDB 
  - Document
- Hbase, 카산드라, 구글빅데이터 등 
  - Wide Column
- Neo4j 
  - Graph

In [1]:
# 135 sqlite3

# brew install sqlite3

import sqlite3

#conn = sqlite3.connect('test_sqlite.db')
conn = sqlite3.connect(':memory:')  # memory db

curs = conn.cursor()

curs.execute('CREATE TABLE persons(id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING)')
conn.commit()

curs.execute('INSERT INTO persons(name) values("Mike")')
curs.execute('INSERT INTO persons(name) values("Nancy")')
curs.execute('INSERT INTO persons(name) values("Jun")')
conn.commit()

curs.execute('UPDATE persons set name = "Michel" WHERE name = "Mike"')
conn.commit()

curs.execute('DELETE FROM persons WHERE name = "Michel"')
conn.commit()

curs.execute('SELECT * FROM persons')
print(curs.fetchall())

curs.close()
conn.close()


[(2, 'Nancy'), (3, 'Jun')]


In [3]:
# 138 mysql

# brew install mysql
# brew services start mysql
# mysql -uroot
# brew services stop mysql

import mysql.connector

# user, password 설정할 것
conn = mysql.connector.connect(host='127.0.0.1', user='root', password='')

cursor = conn.cursor()

# cursor.execute(
#     'CREATE DATABASE test_mysql_database'
# )

cursor.close()
conn.close()


In [32]:
import mysql.connector

# user, password 설정할 것
conn = mysql.connector.connect(host='127.0.0.1', user='root', password='', database='test_mysql_database')

cursor = conn.cursor()

# cursor.execute(
#     'CREATE TABLE persons('
#     'id int NOT NULL AUTO_INCREMENT,'
#     'name VARCHAR(14) NOT NULL,'
#     'PRIMARY KEY(id))'
# )

# cursor.execute('INSERT INTO persons(name) values("MIKE")')

conn.commit()

cursor.execute('SELECT * FROM persons')
for row in cursor:
    print(row)

cursor.execute('UPDATE persons set name = "Michel" where name = "MIKE"')
cursor.execute('DELETE FROM persons where name = "Michel"')
    
cursor.close()
conn.close()

(1, 'MIKE')
(2, 'MIKE')


In [33]:
# 139 SQLAlchemy ORM

# pip install sqlalchemy
# pip install pymysql # mysql 연결형

import sqlalchemy
import sqlalchemy.ext.declarative
import sqlalchemy.orm

engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True)
#engine = sqlalchemy.create_engine('sqlite:///test_sqlite')
#engine = sqlalchemy.create_engine('mysql+pymysql:///test_mysql_databases2')

Base = sqlalchemy.ext.declarative.declarative_base()

class Person(Base):
    __tablename__ = 'persons'
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True, autoincrement=True)
    name = sqlalchemy.Column(sqlalchemy.String(14))
    
Base.metadata.create_all(engine)


Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

p1 = Person(name='Mike')
session.add(p1)
p2 = Person(name='Nancy')
session.add(p2)
p3 = Person(name='Jun')
session.add(p3)

session.commit()

# ORM 선택
p4 = session.query(Person).filter_by(name='Mike').first()
p4.name = 'Michel'
session.add(p4)
session.commit()

# ORM 삭제
p5 = session.query(Person).filter_by(name='Nancy').first()
session.delete(p5)
session.commit()

persons = session.query(Person).all()
for person in persons:
    print(person.id, person.name)

2021-08-23 12:13:36,337 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-23 12:13:36,338 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("persons")
2021-08-23 12:13:36,339 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-23 12:13:36,340 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("persons")
2021-08-23 12:13:36,341 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-23 12:13:36,344 INFO sqlalchemy.engine.Engine 
CREATE TABLE persons (
	id INTEGER NOT NULL, 
	name VARCHAR(14), 
	PRIMARY KEY (id)
)


2021-08-23 12:13:36,345 INFO sqlalchemy.engine.Engine [no key 0.00101s] ()
2021-08-23 12:13:36,347 INFO sqlalchemy.engine.Engine COMMIT
2021-08-23 12:13:36,349 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-23 12:13:36,350 INFO sqlalchemy.engine.Engine INSERT INTO persons (name) VALUES (?)
2021-08-23 12:13:36,351 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ('Mike',)
2021-08-23 12:13:36,352 INFO sqlalchemy.engine.Engine INSERT INTO persons (name) VALUES (

In [37]:
# 140 DBM

import dbm

# with dbm.open('cache', 'c')as db:  # cache.db 파일 생성
#     db['key1'] = 'value1'
#     db['key2'] = 'value2' # byte, string만 작성 가능함

with dbm.open('cache', 'r') as db:
    print(db.get('key1'))


b'value1'


In [53]:
# 141 memcached

# brew install memcached
# memcached -vv (기동)
# brew services start memcached

# pip install python-memcached

import sqlite3
import time

import memcache

db = memcache.Client(['127.0.0.1:11211', ])

db.set('web_page', 'value1', time=1) # 1초 후 사라짐

# print(db.get('web_page'))
# time.sleep(2)
# print(db.get('web_page'))

# db.set('counter', 0)
# db.incr('counter', 1)
# db.incr('counter', 1)
# db.incr('counter', 1)
# db.incr('counter', 1)
# db.incr('counter', 1)
# print(db.get('counter'))

# 초기 설정
conn = sqlite3.connect('test_sqlite.db')
curs = conn.cursor()
# curs.execute(
#     'CREATE TABLE persons('
#     'employ_id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING)')
# curs.execute('INSERT INTO persons(name) values("Mike")')
# conn.commit()
# conn.close()

# memcached에 저장되어 있는지 먼저 확인 후 없으면 db에서 불러옴 (속도!)
def get_employ_id(name):
    employ_id = db.get(name) 
    if employ_id:
        return employ_id
    curs.execute(
        'SELECT * FROM persons WHERE name="{}"'.format(name)
    )
    person = curs.fetchone()
    if not person:
        raise Exception('No employ')
    employ_id, name = person
    db.set(name, employ_id, time=60)
    return employ_id

print(get_employ_id("Mike"))

1


In [59]:
# 142 pickle

import pickle

class T(object):
    
    def __init__(self, name):
        self.name = name

data ={
    'a': [1, 2, 3],
    'b': ('test', 'test'),
    'c': {'key' : 'value'},
    'd': T('test')
}

with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)


with open('data.pickle', 'rb') as f:
    data_loaded = pickle.load(f)
    print(data_loaded['d'].name)

test


In [84]:
# 145 mongodb 

# install mac https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/
# xcode-select --install 
# brew tap mongodb/balrew
# brew install mongodb-community@5.0
# brew services start mongodb-community@5.0
# brew services stop mongodb-community@5.0
# pip install pymongo

import datetime

from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['test_database']

stack1 = {
    'name': 'customer1',
    'pip' : ['python', 'java', 'go'],
    'info' : {'os': 'mac'}, 
    'data' : datetime.datetime.utcnow()
}

stack2 = {
    'name': 'customer2',
    'pip' : ['python', 'java'],
    'info' : {'os': 'windows'}, 
    'data' : datetime.datetime.utcnow()
}

db_stacks = db.stacks
# stack_id = db_stacks.insert_one(stack1).inserted_id
# print(stack_id, type(stack_id))
# print("#################")

# from bson.objectid import ObjectId # 이렇게 넣어줘야 함
# str_stack_id = '61232d745ebbc6429d156320'
# print(db_stacks.find_one({'_id': ObjectId(str_stack_id)}))

# print("#################")

# print(db_stacks.find_one({'name': 'customer1'}))
# print(db_stacks.find_one({'pip': ['python', 'java', 'go']}))

# stack_id = db_stacks.insert_one(stack2).inserted_id
# print(stack_id, type(stack_id))

# print("#################")

# for stack in db_stacks.find():
#     print(stack)
    
# print("#################")
now = datetime.datetime.utcnow()
for stack in db_stacks.find({'data': {'$gt': now}}):
    print(stack)
    
# Update
# db_stacks.find_one_and_update(
#     {'name':'customer1'}, {'$set': {'name': 'YYY'}}
# )
# print(db_stacks.find_one({'name': 'YYY'}))

# Delete
# db_stacks.delete_one({'name': 'YYY'})
# print(db_stacks.find_one({'name': 'YYY'}))



In [85]:
# 146 Hbase

# brew install jenv  java 버전 선택
# brew install openjdk 
# sudo ln -sfn /opt/homebrew/opt/openjdk/libexec/openjdk.jdk /Library/Java/JavaVirtualMachines/openjdk.jdk
# jenv global 17
# jenv versions

# brew install hbase
# brew services start hbase



In [ ]:
# 149 

# neo4j.com
